In [115]:
import numpy as np
import plotly.express as px
import math
import scipy.stats as st
import plotly.figure_factory as ff


$λ$: Taxa de chegada

$μ$: Taxa de atendimento / velocidade do servidor

$ρ$: Percentual de tempo que o sistema permanece ocupado Marcos

$w$: Tempo médio de permanência do cliente no sistema (Fila + Atendimento) OK

$w_q$:Tempo médio de permanência do cliente na fila OK

$L$: Média de clientes no sistema  

$L_q$: Média de clientes na fila

$π_n$: Probabilidade de haver n clientes na fila. 

In [324]:
def simulador(tempo_total, lambda_, mi_):
    n = 0
    tempo_atual = 0
    espera_sitema = 0
    espera_fila = 0
    clientes = 0

    while tempo_atual < tempo_total:
        arrival_time = np.random.exponential(scale=1/lambda_)
        departure_time = np.random.exponential(scale=1/mi_)

        if n == 0 or arrival_time < departure_time:
            tempo_anterior = tempo_atual
            tempo_atual += arrival_time
            espera_sitema+=n*(tempo_atual-tempo_anterior)
            n += 1

            if n > 1:
                espera_fila+=(n-1)*(tempo_atual-tempo_anterior)
                clientes+=1

        else:
            tempo_anterior = tempo_atual
            tempo_atual += departure_time
            espera_sitema+=n*(tempo_atual-tempo_anterior)
            n -= 1

    w = espera_sitema/tempo_atual
    wq = espera_fila/clientes
    return w, wq

In [323]:
sum([simulador(1000, 1, 2)[1] for _ in range(50)])/50

1.0143311589073678

In [298]:
simulador(1000,1,2)

(0.8451167180406701, 0.5982993657701651)

In [98]:
def simulator_confidence_interval(sample_size, conf_level):
    """
    Simula o processo de fila m/m/1 um intervalo de confiança para tempo médio de espera na fila.

    sample_size: tamanho da amostra para a simulação
    confidence_level: nível de confiança desejado para o intervalo de confiança

    Retorna um intervalo de confiança para o tempo médio de espera na fila.
    """
    results = [simulador(1500, 1, 2) for _ in range(sample_size)]

    # Cálculo do intervalo de confiança
    proportion = np.sum(results) / sample_size
    std_error = math.sqrt(proportion * (1 - proportion) / sample_size)
    margin_error = std_error * st.norm.ppf(conf_level)
    lower_bound = proportion - margin_error #limite superior do IC
    upper_bound = proportion + margin_error  # limite inferior do IC

    return (lower_bound, upper_bound), results

In [99]:
intervalo, data = simulator_confidence_interval(1000, 0.95)
print(intervalo)

(0.4725747033560421, 0.5245893329735669)


In [100]:
px.box(data, orientation='h')

In [114]:
hist_data = [data]
group_labels = ['']

fig = ff.create_distplot(hist_data, group_labels, curve_type='normal', bin_size=0.009)
fig.show()

In [107]:
px.histogram(data, nbins=70)